# CAB420, Week 1 Practical - Question 1 Solution

## Combining and Filtering Multiple Datasets

CAB420 Tutorial1.zip contains a number of datasets, split into two directories as follows:
* BOM, which contains Bureau of Meteorology data for Brisbane City from the years 1999-2019. The data is split into three files.
 * IDCJAC0009_040913_1800_Data.csv contains daily rainfall data;
 * IDCJAC0010_040913_1800_Data.csv contains maximum daily temperature data; and
 * IDCJAC0013_040913_1800_Data.csv contains daily solar exposure data.
* BCCCyclewayCounts contains five years data (from 2014-2018) for Brisbane City Council cycleways, with data for each year being in a separate file (i.e. bike-ped-auto-counts-2014.csv contains data for the year 2014).

You are to combine these datasets into a single table using Python (or the programming language of your choice such that):
* You have a single table that spans the time period of the BCCCyclewayCounts data;
* Duplicate information is avoided (i.e. you don’t have multiple date columns, or similar);
* For the cycle way data, only columns that are available in all years data are included in the final table (i.e. if a counter is available in 2014 − 2017, but not 2018, that column should be excluded.

### Suggested Packages

The following packages are suggested, however there are many ways to approach things in python, if you'd rather use different pacakges that's cool too.

In [8]:
# numpy handles pretty much anything that is a number/vector/matrix/array
import numpy as np
# pandas handles dataframes
import pandas as pd
# matplotlib emulates Matlabs plotting functionality
import matplotlib.pyplot as plt
# stats models is a package that is going to perform the regression analysis
from statsmodels import api as sm
from scipy import stats
from sklearn.metrics import mean_squared_error
# os allows us to manipulate variables on out local machine, such as paths and environment variables
import os
# self explainatory, dates and times
from datetime import datetime, date
# a helper package to help us iterate over objects
import itertools

from typing import List

/opt/conda/envs/cab420-env/lib/python3.9/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


### Step 1: Load Weather Data

You have three weather files to load. Each of these will have more data than we do in the cycling files, so only select data in the same data range as we have for the cycle counts (2014-2018).

You may also wish to convert the data information that is in this data into a date object to make it easier to work with. If you wish to do this, you can convert the three date values in each row (Year, Month, Day) to a datetime object with the following function. You could use the ``apply()`` function that operates over a pandas dataframe to apply this to your tables.

Ideally, it'd be good to merge these into one overall weather table. It's suggested to do a quick visualisation of your BOM data after it's loaded. The ``head()`` function is good for this. You may then want to remove some redundant columns. ``drop()`` can be used to achieve this.

In [9]:
os.path.join("IDCJAC0009_040913_1800_Data.csv")

'IDCJAC0009_040913_1800_Data.csv'

In [57]:
def load_weather_data(rootPath=""):
    boms = {
        "1":pd.read_csv(os.path.join(rootPath, "IDCJAC0009_040913_1800_Data.csv")),
        "2":pd.read_csv(os.path.join(rootPath, "IDCJAC0010_040913_1800_Data.csv")),
        "3": pd.read_csv(os.path.join(rootPath, "IDCJAC0016_040913_1800_Data.csv"))
        }
    for key in boms:
        bom = boms[key]
        # convert ["Year", "Month", "Day"] to datetime obj
        bom["Date"] = bom.apply(lambda row: datetime(row.Year, row.Month, row.Day), axis=1)
        
        # drop redundant columns
        redundant_columns = ["Product code",
                         "Year","Month","Day",
                         "Bureau of Meteorology station number",
                         "Quality"
                         ]
        bom = bom.drop(columns=redundant_columns, errors="ignore")
        
        # 2014-2018
        boms[key] = bom[ ( 2014 <= bom["Date"].dt.year) & (bom["Date"].dt.year <= 2018)]

    # merge them into one table
    # concatenate columns
    bom_combined =  pd.merge(boms["1"], boms["2"], on="Date")
    bom_combined =  pd.merge(bom_combined, boms["3"], on="Date")


    return bom_combined


bom = load_weather_data("BOM")
bom["Date"]


0      2014-01-01
1      2014-01-02
2      2014-01-03
3      2014-01-04
4      2014-01-05
          ...    
1821   2018-12-27
1822   2018-12-28
1823   2018-12-29
1824   2018-12-30
1825   2018-12-31
Name: Date, Length: 1826, dtype: datetime64[ns]

In [ ]:
def create_date(row):
    # create string with date it format Y:m:d
  return datetime.strptime('{:04d}/{:02d}/{:02d}'.format(row.Year, row.Month, row.Day),
                           '%Y/%m/%d')

### Step 2: Load BCC Data

Now we need to load the BCC data. This will follow a broadly similar path to the BOM data:
* Load the individual files
* Convert the date. This data already has a date column, but you should check the format to make sure it's being parsed as a date object and not a string. The ``to_datetime()`` within the pandas dataframe class could be of use.
* Merge the tables. To do this, you will need to look at what columns are in common between the five tables. You can create a list of column names, and use the intersection of this list to find the set of common columns.
* Pull out the common columns from the individual tables, and merge the final results

As with the BOM data, inspect the data after it has been merged.

In [51]:
def load_bcc(rootPath=""):
    bcc_data = {}
    start_year = 2014
    end_year = 2018

    all_columns = set()
    for year in range(start_year, end_year+1):
        bcc_data[year] = pd.read_csv(os.path.join(rootPath,f"bike-ped-auto-counts-{year}.csv"))
        # convert to datetime
        bcc_data[year]["Date"] = pd.to_datetime(bcc_data[year]["Date"],format="%d/%m/%Y")
        all_columns.update(bcc_data[year].columns.values)

    # find what columns are in common between the five tables
    common_columns = all_columns.intersection(*bcc_data.values())
    for key in bcc_data:
        bcc_data[key] = bcc_data[key][list(common_columns)]

    bcc = pd.concat([*bcc_data.values()])
    return bcc


bcc = load_bcc("BCCCyclewayCounts")


In [56]:
first       = {"c", "d", "e"}
second      = {"c", "f", "g"}
all_columns = {*first, *second}

result = all_columns.intersection(first, second)

print(result)

{'c', 'd', 'f', 'e', 'g'}
{'c'}


### Step 3: Merge the Data

Here, we can use the pandas ``merge()`` function to merge our two dataframes. Consider merging them based on the ``Date`` columns to make sure that entries line up.

Visualise the merged dataset, and consider if there are any other columns that could be removed.

Finally, you may want to save the dataset using the ``to_csv()`` function in the pandas dataframe object.

In [52]:
combined = pd.merge(bom, bcc, on="Date")
combined

,Rainfall amount (millimetres),Period over which rainfall was measured (days),Date,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Daily global solar exposure (MJ/m*m),Bicenntenial Bikeway,Kedron Brook Bikeway Mitchelton Pedestrian Outbound,Unnamed: 1,Riverwalk Cyclists Inbound,...,Riverwalk Pedestrians Outbound,North Brisbane Bikeway Mann Park Windsor Pedestrian Inbound,Story Bridge West Cyclists Outbound,Bicentennial Bikeway Pedestrians Outbound,Granville Street Bridge Pedestrians Outbound,Granville Street Bridge Cyclists Outbound,Granville Street Bridge Pedestrians Inbound,Jack Pesch Bridge Cyclists Outbound,Ekibin Park Pedestrians Inbound,Ekibin Park Pedestrians Outbound
0,0.0,1.0,2014-01-01,30.6,1.0,31.2,3333.0,NaN,0:00,NaN,...,NaN,NaN,0.0,1326.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,2014-01-02,31.8,1.0,23.4,4863.0,NaN,0:00,NaN,...,NaN,NaN,0.0,2133.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,2014-01-03,34.5,1.0,29.6,3905.0,NaN,0:00,NaN,...,NaN,NaN,0.0,1867.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,1.0,2014-01-04,38.7,1.0,30.5,3066.0,NaN,0:00,NaN,...,NaN,NaN,0.0,1522.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1.0,2014-01-05,33.6,1.0,15.7,4550.0,NaN,0:00,NaN,...,NaN,NaN,0.0,2102.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,0.0,1.0,2018-12-27,29.5,1.0,29.7,3489.0,93.0,00:00,NaN,...,NaN,176.0,NaN,534.0,62.0,84.0,90.0,173.0,261.0,283.0
1822,0.0,1.0,2018-12-28,29.7,1.0,28.6,3521.0,129.0,00:00,NaN,...,NaN,164.0,NaN,537.0,57.0,64.0,70.0,177.0,247.0,235.0
1823,0.0,1.0,2018-12-29,30.5,1.0,29.8,3480.0,117.0,00:00,NaN,...,NaN,131.0,NaN,527.0,44.0,40.0,80.0,259.0,424.0,367.0
1824,0.0,1.0,2018-12-30,30.6,1.0,27.9,3501.0,116.0,00:00,NaN,...,NaN,146.0,NaN,531.0,48.0,39.0,87.0,199.0,221.0,259.0


In [53]:
combined.columns.values

array(['Rainfall amount (millimetres)',
       'Period over which rainfall was measured (days)', 'Date',
       'Maximum temperature (Degree C)',
       'Days of accumulation of maximum temperature',
       'Daily global solar exposure (MJ/m*m)', 'Bicenntenial Bikeway',
       'Kedron Brook Bikeway Mitchelton Pedestrian Outbound',
       'Unnamed: 1', 'Riverwalk Cyclists Inbound',
       'North Brisbane Bikeway Mann Park Windsor Pedestrian Outbound',
       'Kedron Brook Bikeway Lutwyche Cyclists Outbound',
       'Kedron Brook Bikeway Lutwyche',
       'Kedron Brook Bikeway Lutwyche Pedestrians Inbound',
       'North Brisbane Bikeway Mann Park Windsor Cyclists Inbound',
       'Granville Street Bridge Cyclists Inbound',
       'Kedron Brook Bikeway Mitchelton',
       'Jack Pesch Bridge Cyclists Inbound',
       'Ekibin Park Cyclists Inbound',
       'Schulz Canal Bridge Cyclists Outbound',
       'Bishop Street Pedestrians Outbound',
       'Kedron Brook Bikeway Lutwyche Cyclists In

In [54]:
combined.to_csv("out.csv")